## Taller N°1 : Aplicacion de Algoritmos de B´usqueda

* Profesor: Jorge Ivan Padilla Buritica
* Estudiante: Juan Felipe Cardona Arango 
* Maestria en Ciencia de Datos y Analitica
* Curso Fundamentos de Inteligencia Artificial
* 2025-02


## Preparacion del entorno de trabajo

In [51]:
import math
import heapq
import time
from utils import reconstruct, compute_hop_dist


# Punto 1:Optimizacion de rutas rurales (Logistica)

## Pregunta orientadora
**¿Cómo llego del nodo de inicio al nodo final gastando lo menos posible?**

---

## Algoritmos usados
- **UCS (Búsqueda de Costo Uniforme)**: explora rutas siempre expandiendo el camino más barato encontrado hasta ahora. Garantiza el camino óptimo si los pesos son positivos.
- **A***: es parecido a UCS, pero usa una **heurística** que le da una idea de qué tan cerca está del destino. Esto le permite ser más rápido sin perder la exactitud si la heurística está bien diseñada.

---

## ¿Qué es la heurística?
Una heurística es una **estimación** del costo que falta para llegar al destino. Por ejemplo, si conocemos las coordenadas de cada punto, podemos usar la **distancia en línea recta** (distancia euclidiana) como estimación.

> Si la línea recta siempre es igual o menor que cualquier ruta real, no se sobreestima el costo y A* seguirá encontrando la mejor ruta.

---

## Solución propuesta
1. Representar el mapa como un grafo con nodos, conexiones y pesos.
2. Elegir el nodo inicial y el nodo destino.
3. Seleccionar el algoritmo (UCS o A*) y, si es A*, definir la heurística.
4. Explorar el grafo paso a paso, comparando costos acumulados.
5. Cuando se llega al destino, devolver el camino encontrado y su costo total.

---

## Medición
Para evaluar un algoritmo no basta con que encuentre el camino óptimo; también importa **el esfuerzo que le costó encontrarlo**.

Por eso medimos:
- **Costo total** del camino encontrado.
- **Nodos expandidos** durante la búsqueda.
- **Profundidad** de la solución (número de pasos o aristas).
- **Tiempo de ejecución** (en milisegundos).



## Codigo

In [2]:
from __future__ import annotations
from typing import Dict, Tuple, Callable, List, Any
import math
import time
import heapq
import networkx as nx


In [3]:
# Runner genérico de algoritmos
from typing import Callable, Dict, Any
def run_compare_algorithms(algos: Dict[str, Callable[[], Any]]):
    """Ejecuta funciones de búsqueda y recopila métricas comunes."""
    table = []
    for name, func in algos.items():
        t0 = time.time()
        res = func()
        t1 = time.time()
        if hasattr(res, 'as_dict'):
            data = res.as_dict()
        elif isinstance(res, dict):
            data = res
        elif isinstance(res, tuple) and len(res)==5:
            path, cost, expd, depth, tms = res
            data = {'path': path, 'cost': cost, 'expanded': expd, 'depth': depth, 'runtime_ms': tms}
        else:
            data = {'result': res}
        data['algo'] = name
        data.setdefault('runtime_ms', round((t1-t0)*1000,3))
        table.append(data)
    return table


In [4]:
if __name__ == '__main__':
    G, s, t = build_sample_graph()
    algos = {
        'UCS': lambda: ucs(G, s, t),
        'A* h=0': lambda: astar(G, s, t, h_zero(t)),
        'A* h=euclid': lambda: astar(G, s, t, h_euclid(G, t)),
        'A* h=1.3*euclid': lambda: astar(G, s, t, h_scaled(G, t, alpha=1.3))
    }
    table = run_compare_algorithms(algos)
    for row in table:
        print(row)


{'algo': 'UCS', 'heuristic': 'h=0', 'path': ['A', 'B', 'C', 'F'], 'cost': 6.708203932499369, 'expanded': 6, 'depth': 3, 'runtime_ms': 0.15}
{'algo': 'A*', 'heuristic': 'h=0', 'path': ['A', 'B', 'C', 'F'], 'cost': 6.708203932499369, 'expanded': 6, 'depth': 3, 'runtime_ms': 0.022}
{'algo': 'A*', 'heuristic': 'h=euclid', 'path': ['A', 'B', 'C', 'F'], 'cost': 6.708203932499369, 'expanded': 5, 'depth': 3, 'runtime_ms': 0.019}
{'algo': 'A*', 'heuristic': 'h=1.3*euclid', 'path': ['A', 'B', 'C', 'F'], 'cost': 6.708203932499369, 'expanded': 4, 'depth': 3, 'runtime_ms': 0.015}


 Interpretación de resultados y comparación de algoritmos
A continuación se comentan las métricas obtenidas en el ejemplo para cada algoritmo:

 Resultados obtenidos
| Algoritmo | Heurística       | Camino encontrado       | Costo   | Nodos expandidos | Profundidad | Tiempo (ms) |
|-----------|------------------|------------------------|---------|------------------|-------------|-------------|
| UCS       | h=0              | A → B → C → F           | 6.7082  | 6                | 3           | 0.150       |
| A*        | h=0              | A → B → C → F           | 6.7082  | 6                | 3           | 0.022       |
| A*        | h=euclid         | A → B → C → F           | 6.7082  | 5                | 3           | 0.019       |
| A*        | h=1.3×euclid     | A → B → C → F           | 6.7082  | 4                | 3           | 0.015       |

---

**Interpretación:**
- Todas las configuraciones encontraron el mismo camino óptimo con costo ≈ 6.71.
- Usar una heurística informativa (euclidiana) redujo la cantidad de nodos expandidos y el tiempo de ejecución frente a UCS.
- La heurística escalada (1.3×euclid) aceleró aún más la búsqueda, aunque en grafos más complejos podría perder optimalidad.



## Respuestas a Preguntas Orientadoras

**1) ¿Cómo afecta la calidad de la heurística?**
> Una heurística más informativa (euclidiana) reduce el número de nodos expandidos y el tiempo de ejecución sin perder optimalidad. La heurística escalada (1.3×euclid) reduce aún más las expansiones, pero podría sacrificar optimalidad en casos más complejos.

**2) ¿Es pertinente la euclidiana?**
> Sí, porque el peso de las aristas se calculó como distancia euclidiana. Esto garantiza que la heurística sea admisible y consistente.

**3) ¿Flexibilidad ante nuevas rutas o restricciones?**
> El modelo permite añadir o quitar aristas y ajustar pesos fácilmente. Los algoritmos pueden recalcular rutas óptimas sin modificar la lógica interna.

**4) ¿Escalabilidad si el grafo crece 10× en tamaño?**
> UCS y A* seguirán funcionando, pero el tiempo y memoria crecerán. Una heurística de calidad (como la euclidiana) será clave para reducir expansiones. En grafos muy grandes se recomienda optimizar con técnicas como búsqueda bidireccional o preprocesamiento de rutas.

---

# Punto 2: Red de metro (transporte)
Se proporciona un grafo no ponderado de estaciones de metro. Compare las soluciones usando **BFS** (Breadth-First Search) e **IDS** (Iterative Deepening Search) para ir de la estación A a la estación J.

**Cuestiones críticas**:
- ¿Qué pasa si el sistema crece a 100 nodos? ¿Cuál es el impacto en complejidad temporal?
- ¿Puede automatizarse la elección del mejor algoritmo según el tamaño de la red?

Cada ejercicio debe incluir:
- Análisis del problema y del tipo de búsqueda.
- Implementación en Python y explicación del código.
- Medidas de desempeño (tiempo, nodos expandidos, profundidad y costo).
- Evaluación de complejidad temporal y espacial.
- Rutinas automáticas de comparación entre algoritmos.
- Reflexión crítica sobre eficiencia y aplicabilidad.


## Codigo

In [1]:
"""
Comparativa de algoritmos de búsqueda en una red de metro (A*, BFS e IDS)
--------------------------------------------------------------------------

Este script implementa y compara tres enfoques clásicos de búsqueda:
  - A*: óptimo en COSTO (tiempo) con heurística admisible.
  - BFS (Breadth-First Search): óptimo en NÚMERO DE ARISTAS (hops) cuando los pesos son iguales.
  - IDS (Iterative Deepening Search): óptimo en NÚMERO DE ARISTAS con baja memoria, pero más re-expansiones.

Incluye:
  1) Construcción de dos grafos de prueba:
     - Pequeño (A..J) con pesos heterogéneos y atajos (bypasses).
     - Grande (~100 nodos) como grilla 10x10 con pesos aleatorios.
  2) Implementaciones propias de A*, BFS e IDS (sin depender de los equivalentes de NetworkX).
  3) Métricas de desempeño: tiempo, nodos expandidos, profundidad (hops) y costo total (minutos).
  4) Heurística admisible para A*: distancia en línea recta / v_max.
  5) Selector automático de algoritmo por reglas sencillas.
"""

import time
import math
import random
from collections import deque
from typing import Dict, List, Optional, Tuple

import networkx as nx
import pandas as pd


# ---------------------------------------------------------------------
# Utilidades
# ---------------------------------------------------------------------



def path_cost(G: nx.Graph, path: List[str]) -> float:
    """Calcula el costo total (minutos) de una ruta dada."""
    if not path or len(path) == 1:
        return 0.0
    total = 0.0
    for u, v in zip(path[:-1], path[1:]):
        total += G[u][v].get('weight', 1.0)
    return total


# ---------------------------------------------------------------------
# Construcción de grafos
# ---------------------------------------------------------------------

def build_metro_graph_small() -> Tuple[nx.Graph, str, str]:
    """Crea un grafo pequeño de prueba (A..J + rama E1-F1)."""
    G = nx.Graph()
    coords = {
        'A': (0, 0), 'B': (2, 0), 'C': (4, 0), 'D': (6, 0),
        'E': (8, 0), 'F': (10, 0), 'G': (12, 0), 'H': (14, 0),
        'I': (16, 0), 'J': (18, 0),
        'E1': (8, -2), 'F1': (10, -2),
    }
    for s, pos in coords.items():
        G.add_node(s, pos=pos)
    edges = [
        ('A','B',3), ('B','C',6), ('C','D',3), ('D','E',5),
        ('E','F',4), ('F','G',6), ('G','H',3), ('H','I',4), ('I','J',5),
        ('B','D',4), ('E','G',5), ('G','I',6),
        ('E','E1',2), ('E1','F1',3), ('F1','F',2)
    ]
    for u, v, w in edges:
        G.add_edge(u, v, weight=w)
    return G, 'A', 'J'


def build_grid_graph(n_rows: int = 10, n_cols: int = 10, weight_range=(2, 8)) -> Tuple[nx.Graph, str, str]:
    """Crea un grafo tipo grilla con pesos aleatorios."""
    G = nx.Graph()
    for r in range(n_rows):
        for c in range(n_cols):
            name = f"r{r}c{c}"
            G.add_node(name, pos=(c*2, r*2))
    for r in range(n_rows):
        for c in range(n_cols):
            u = f"r{r}c{c}"
            if r + 1 < n_rows:
                v = f"r{r+1}c{c}"
                G.add_edge(u, v, weight=random.randint(*weight_range))
            if c + 1 < n_cols:
                v = f"r{r}c{c+1}"
                G.add_edge(u, v, weight=random.randint(*weight_range))
    return G, "r0c0", f"r{n_rows-1}c{n_cols-1}"


# ---------------------------------------------------------------------
# Heurística para A*
# ---------------------------------------------------------------------

def h_metro(G: nx.Graph, goal: str, vmax: float = 0.667):
    """Heurística: distancia euclidiana / velocidad máxima."""
    xg, yg = G.nodes[goal]['pos']
    def h(n: str) -> float:
        x, y = G.nodes[n]['pos']
        dist = math.hypot(x - xg, y - yg)
        return dist / vmax
    return h


# ---------------------------------------------------------------------
# Algoritmos de búsqueda
# ---------------------------------------------------------------------

import heapq

def astar_metro(G: nx.Graph, start: str, goal: str, hfunc):
    """Implementación de A*."""
    openq = []
    heapq.heappush(openq, (hfunc(start), start))
    g = {start: 0.0}
    parent = {start: None}
    expanded = 0
    closed = set()
    while openq:
        _, n = heapq.heappop(openq)
        if n in closed:
            continue
        closed.add(n)
        expanded += 1
        if n == goal:
            path = reconstruct(parent, goal)
            return path, path_cost(G, path), expanded, len(path)-1
        for nbr in G.neighbors(n):
            w = G[n][nbr].get('weight', 1.0)
            tentative = g[n] + w
            if tentative < g.get(nbr, float('inf')):
                g[nbr] = tentative
                parent[nbr] = n
                heapq.heappush(openq, (tentative + hfunc(nbr), nbr))
    return [], float('inf'), expanded, None


def bfs_unweighted(G: nx.Graph, start: str, goal: str):
    """Búsqueda en anchura (óptima en hops si pesos iguales)."""
    q = deque([start])
    parent = {start: None}
    visited = {start}
    expanded = 0
    while q:
        n = q.popleft()
        expanded += 1
        if n == goal:
            path = reconstruct(parent, goal)
            return path, path_cost(G, path), expanded, len(path)-1
        for nbr in G.neighbors(n):
            if nbr not in visited:
                visited.add(nbr)
                parent[nbr] = n
                q.append(nbr)
    return [], float('inf'), expanded, None


def dls(G, current, goal, limit, parent, visited, expanded):
    """Búsqueda en profundidad limitada."""
    expanded[0] += 1
    if current == goal:
        return True
    if limit == 0:
        return False
    for nbr in G.neighbors(current):
        if nbr not in visited:
            visited.add(nbr)
            parent[nbr] = current
            if dls(G, nbr, goal, limit-1, parent, visited, expanded):
                return True
    return False


def ids_unweighted(G, start, goal, max_depth=1000):
    """Búsqueda con profundización iterativa."""
    expanded_total = 0
    for depth in range(max_depth+1):
        parent = {start: None}
        visited = {start}
        expanded = [0]
        found = dls(G, start, goal, depth, parent, visited, expanded)
        expanded_total += expanded[0]
        if found:
            path = reconstruct(parent, goal)
            return path, path_cost(G, path), expanded_total, len(path)-1, depth
    return [], float('inf'), expanded_total, None, None


# ---------------------------------------------------------------------
# Ejecución y comparación
# ---------------------------------------------------------------------

def run_experiments():
    # --- Grafo pequeño ---
    Gs, s, t = build_metro_graph_small()
    hs = h_metro(Gs, t)
    t0 = time.perf_counter()
    pa, ca, ea, da = astar_metro(Gs, s, t, hs)
    t1 = time.perf_counter()
    t2 = time.perf_counter()
    pb, cb, eb, db = bfs_unweighted(Gs, s, t)
    t3 = time.perf_counter()
    t4 = time.perf_counter()
    pi, ci, ei, di, limi = ids_unweighted(Gs, s, t, max_depth=50)
    t5 = time.perf_counter()
    df_small = pd.DataFrame([
        {"Algoritmo": "A*", "Ruta": "→".join(pa), "Costo (min)": round(ca,2),
         "Profundidad": da, "Nodos expandidos": ea, "Tiempo (ms)": round((t1-t0)*1000,3)},
        {"Algoritmo": "BFS", "Ruta": "→".join(pb), "Costo (min)": round(cb,2),
         "Profundidad": db, "Nodos expandidos": eb, "Tiempo (ms)": round((t3-t2)*1000,3)},
        {"Algoritmo": "IDS", "Ruta": "→".join(pi), "Costo (min)": round(ci,2),
         "Profundidad": di, "Nodos expandidos": ei, "Tiempo (ms)": round((t5-t4)*1000,3)}
    ])
    print("\n=== Resultados - Grafo A..J ===")
    print(df_small.to_string(index=False))

    # --- Grafo grande ---
    Gg, sg, tg = build_grid_graph(10, 10, (2,8))
    hg = h_metro(Gg, tg)
    tb0 = time.perf_counter()
    pa_b, ca_b, ea_b, da_b = astar_metro(Gg, sg, tg, hg)
    tb1 = time.perf_counter()
    tb2 = time.perf_counter()
    pb_b, cb_b, eb_b, db_b = bfs_unweighted(Gg, sg, tg)
    tb3 = time.perf_counter()
    tb4 = time.perf_counter()
    pi_b, ci_b, ei_b, di_b, limi_b = ids_unweighted(Gg, sg, tg, max_depth=200)
    tb5 = time.perf_counter()
    df_big = pd.DataFrame([
        {"Algoritmo": "A*", "Long. ruta": len(pa_b), "Costo (min)": round(ca_b,2),
         "Profundidad": da_b, "Nodos expandidos": ea_b, "Tiempo (ms)": round((tb1-tb0)*1000,3)},
        {"Algoritmo": "BFS", "Long. ruta": len(pb_b), "Costo (min)": round(cb_b,2),
         "Profundidad": db_b, "Nodos expandidos": eb_b, "Tiempo (ms)": round((tb3-tb2)*1000,3)},
        {"Algoritmo": "IDS", "Long. ruta": len(pi_b), "Costo (min)": round(ci_b,2),
         "Profundidad": di_b, "Nodos expandidos": ei_b, "Tiempo (ms)": round((tb5-tb4)*1000,3)}
    ])
    print("\n=== Resultados - Grafo 10x10 (~100 nodos) ===")
    print(df_big.to_string(index=False))


if __name__ == "__main__":
    run_experiments()



=== Resultados - Grafo A..J ===
Algoritmo                Ruta  Costo (min)  Profundidad  Nodos expandidos  Tiempo (ms)
       A*       A→B→D→E→G→I→J         28.0            6                 9        0.065
      BFS       A→B→D→E→G→I→J         28.0            6                12        0.015
      IDS A→B→C→D→E→F→G→H→I→J         39.0            9                65        0.028

=== Resultados - Grafo 10x10 (~100 nodos) ===
Algoritmo  Long. ruta  Costo (min)  Profundidad  Nodos expandidos  Tiempo (ms)
       A*          19         60.0           18                78        0.250
      BFS          19         86.0           18               100        0.049
      IDS          41        189.0           40              2072        0.643


## Respuestas a Preguntas Orientadoras

Escalamiento a ~100 nodos: qué ocurre y cómo decidir el algoritmo
1) ¿Qué pasa si el sistema crece a ~100 nodos?
Tus propios resultados ya lo muestran (grafo 10×10 ≈100 nodos):

A*: nodos expandidos pasan de 9 → 78 (×8.7); tiempo 0.065 → 0.250 ms (×3.8).
Lectura: con una heurística informativa, A* escala mucho mejor: expande regiones “cerca” del óptimo.

BFS: 12 → 100 (×8.3); 0.015 → 0.049 ms (×3.3).
Lectura: en rejillas, BFS termina explorando casi toda la “bola” de radio d; en 10×10 se acerca a V.

IDS: 65 → 2072 (×31.9); 0.028 → 0.643 ms (×23).
Lectura: el costo de re‑expansión por profundidades sucesivas explota rápido.

Conclusión empírica: al crecer el grafo, IDS se degrada más; BFS crece casi lineal con V en rejillas poco profundas; A* mantiene buen comportamiento si la heurística es consistente e informativa.



2) Impacto en complejidad temporal
Sea b el factor de ramificación y d la profundidad de la solución.

BFS (no ponderado): tiempo O(b^d), espacio O(b^d). En rejillas con b≈4 y d≈O(√V), el trabajo puede crecer cerca de O(V) en la práctica.

IDS (no ponderado): tiempo O(b^d) pero con constante peor (re‑explora niveles); espacio O(bd).

Uniform‑Cost / Dijkstra (ponderado, no heurístico): O(E log V) con heap binario; en rejillas E=O(V) ⇒ O(V log V).

A* (ponderado, heurística admisible y consistente): peor caso exponencial, pero en la práctica ≈ trabajo de UCS limitado a nodos con f(n)=g(n)+h(n) ≤ C*. Si h es buena, la expansión se acerca a O(óptimo); en rejillas con Manhattan, el crecimiento suele ser cuasi‑lineal en V para rutas “rectas”.

# Punto 3: Filogenia (biología)

Un **árbol filogenético** modela la historia evolutiva de un conjunto de especies.
Cada nodo representa un ancestro y las hojas son especies actuales.
El objetivo es hallar el **ancestro común más reciente (LCA)** entre dos especies dadas.

Utilizamos **búsqueda en profundidad limitada (DLS)** para obtener las rutas
desde la raíz hacia cada especie restringiendo la profundidad.
Esto es útil cuando la altura del árbol es conocida o acotada.
Como referencia se emplea BFS para contrastar el número de nodos expandidos.


## Codigo

In [ ]:
if __name__ == '__main__':
    # Árbol de ejemplo:
    #         Root
    #        /    \ 
    #    Clade1  Clade2
    #     /  \     /  \ 
    #   SppA SppB SppC SppD
    T = PhyloTree('Root')
    T.add_edge('Root', 'Clade1'); T.add_edge('Root', 'Clade2')
    T.add_edge('Clade1', 'SppA'); T.add_edge('Clade1', 'SppB')
    T.add_edge('Clade2', 'SppC'); T.add_edge('Clade2', 'SppD')
    pares = [('SppA','SppB'), ('SppA','SppC'), ('SppC','SppD')]
    for u, v in pares:
        print(f'
Par {u}-{v}:')
        rows = run_compare_algorithms({
            'DLS': lambda u=u, v=v: lca_with_method(T, u, v, method='DLS', max_depth=3),
            'BFS': lambda u=u, v=v: lca_with_method(T, u, v, method='BFS')
        })
        for r in rows:
            print(r)



Par SppA-SppB (DLS):
{'tiempo_ms': 0.026, 'expandidos_u': 7, 'expandidos_v': 8, 'profundidad_u': 2, 'profundidad_v': 2, 'ruta_u': ['Root', 'Clade1', 'SppA'], 'ruta_v': ['Root', 'Clade1', 'SppB'], 'LCA': 'Clade1'}
Par SppA-SppB (BFS):
{'tiempo_ms': 0.028, 'expandidos_u': 4, 'expandidos_v': 5, 'profundidad_u': 2, 'profundidad_v': 2, 'ruta_u': ['Root', 'Clade1', 'SppA'], 'ruta_v': ['Root', 'Clade1', 'SppB'], 'LCA': 'Clade1'}

Par SppA-SppC (DLS):
{'tiempo_ms': 0.01, 'expandidos_u': 7, 'expandidos_v': 10, 'profundidad_u': 2, 'profundidad_v': 2, 'ruta_u': ['Root', 'Clade1', 'SppA'], 'ruta_v': ['Root', 'Clade2', 'SppC'], 'LCA': 'Root'}
Par SppA-SppC (BFS):
{'tiempo_ms': 0.011, 'expandidos_u': 4, 'expandidos_v': 6, 'profundidad_u': 2, 'profundidad_v': 2, 'ruta_u': ['Root', 'Clade1', 'SppA'], 'ruta_v': ['Root', 'Clade2', 'SppC'], 'LCA': 'Root'}

Par SppC-SppD (DLS):
{'tiempo_ms': 0.01, 'expandidos_u': 10, 'expandidos_v': 11, 'profundidad_u': 2, 'profundidad_v': 2, 'ruta_u': ['Root', 'Clade2',

## Respuestas a Preguntas Orientadoras

Respuestas técnicas a partir de tus resultados
1) ¿Cómo afecta la elección de profundidad límite?
Si el límite < profundidad real del objetivo, IDS/DLS no encuentra la ruta y el LCA queda None → falso negativo.

Si el límite = profundidad real, encuentra la ruta con mínimo trabajo extra.

Si el límite > profundidad real, IDS re-expande niveles innecesarios; el costo crece por la reexploración de prefijos en cada iteración.

En tus números se ve el patrón esperado: BFS expande menos que IDS (p. ej., SppA–SppB: 4–5 vs 7–8) porque IDS paga la penalización de re-expansión aun con un árbol pequeño.

Recomendación práctica:

Adaptar el límite dinámicamente (L=0…∞ hasta hallar o tope razonable).

O mejor, estimar un tope con un BFS rápido desde la raíz que te entregue depth[nodo] y así fijar limit = max(depth[u], depth[v]).

2) ¿Podría automatizarse este análisis para múltiples pares?
Sí, y con fuerte ahorro de cómputo si separas pre-cómputo de consultas:

Pre-cómputo único (una sola pasada):

Ejecuta BFS desde la raíz para obtener parent[] y depth[] de todos los nodos (y, si quieres, la ruta se reconstruye en O(h)).

Opcional: construye estructura de LCA rápida:

Binary Lifting (preproceso O(N log N), consulta O(log N)), o

Euler Tour + RMQ (preproceso O(N)–O(N log N), consulta O(1)–O(log N)), o

Tarjan offline (si tienes todos los pares de antemano), casi lineal.

Loop de pares:

Para cada par (u, v), responde el LCA con la estructura ya precomputada (sin volver a explorar el árbol).

Mantén métrica por par (tiempo, profundidad, etc.).

Si necesitas comparar con IDS/DLS “por consigna”, ejecútalo solo para el par, pero reporta a la vez el baseline óptimo (LCA por parent/depth) y el costo de re-expansión de IDS para efecto didáctico.

Resultado: pasas de “reexplorar el árbol por par” a O(1)–O(log N) por consulta tras un único pre-cómputo.

3) ¿Qué tan escalable es con árboles reales (cientos de especies)?
Para N ≈ 10²–10³:

Un BFS único es O(N) y barato en memoria.

IDS por par ya no escala bien si haces muchas consultas (re-explora prefijos).

Con Binary Lifting o Euler+RMQ, miles de consultas de LCA son inmediatas.

Para Q pares:

Sin preproceso (IDS/BFS por par): coste ~ O(Q·N) en el peor caso.

Con preproceso:

Binary Lifting: O(N log N) + O(Q log N).

Euler+RMQ (Sparse Table): O(N log N) + O(1) por consulta.

Tarjan offline: O(N + Q α(N)) ~ lineal si conoces todos los pares antes.

Detalles de implementación en Python:

Evita recursión profunda en DLS para árboles altos → usa pila explícita.

Ordena children si quieres reproducibilidad.

Valida que no haya ciclos ni nodos con >1 padre.

Si el árbol viene de Newick/PhyloXML, puedes parsear con Biopython y luego construir tus arrays parent/depth.

# Punto 4: Evacuación de emergencia (infraestructura)

En este escenario cada arista posee un **nivel de riesgo** basado en reportes de siniestralidad y condiciones viales.
La suma de dichos riesgos sobre una ruta determina su peligrosidad total, por lo que buscamos el camino de **menor riesgo acumulado**.

Se utiliza **Uniform Cost Search (UCS)** porque garantiza encontrar la ruta más segura cuando los costos son no negativos.
A* puede acelerar la búsqueda si dispone de una heurística admisible de riesgo; de lo contrario se comporta igual que UCS,
por eso UCS es la elección natural frente a métricas como distancia o tiempo.


## Codigo

In [50]:
# -*- coding: utf-8 -*-
"""
Evacuación de emergencia: ruta más segura con UCS (Uniform Cost Search).
- Pesos = nivel de RIESGO (no distancia).
- Soporta múltiples salidas, cierres de tramos y escenarios con incertidumbre (Monte Carlo).
- Métricas: tiempo, nodos expandidos, profundidad (hops), riesgo acumulado.
"""
import heapq
import math
import random
import time
from collections import deque
from typing import Dict, List, Tuple, Optional, Set

class Graph:
    def __init__(self):
        self.adj: Dict[str, List[Tuple[str, Dict]]] = {}

    def add_edge(self, u: str, v: str, risk: float, closed: bool=False, mu: Optional[float]=None, sigma: Optional[float]=None):
        self.adj.setdefault(u, []).append((v, {'risk': risk, 'closed': closed, 'mu': mu, 'sigma': sigma}))
        self.adj.setdefault(v, []).append((u, {'risk': risk, 'closed': closed, 'mu': mu, 'sigma': sigma}))

    def neighbors(self, u: str):
        return self.adj.get(u, [])

    def set_closed(self, u: str, v: str, closed: bool=True):
        for i, (w, attrs) in enumerate(self.adj.get(u, [])):
            if w == v:
                attrs['closed'] = closed
        for i, (w, attrs) in enumerate(self.adj.get(v, [])):
            if w == u:
                attrs['closed'] = closed

def build_small_graph() -> Tuple[Graph, str, Set[str]]:
    G = Graph()
    edges = [
        ('A','B',3), ('B','C',6), ('C','D',3), ('D','E',5),
        ('E','F',4), ('F','G',7), ('G','H',3), ('H','I',6), ('I','J',4),
        ('B','D',2), ('E','G',5), ('G','I',5), ('C','F',10), ('F','I',9)
    ]
    for u,v,r in edges:
        G.add_edge(u,v,risk=r)
    return G, 'A', {'J'}

def build_grid_graph(n: int=10, rng=(1,10), p_block=0.05) -> Tuple[Graph, Tuple[int,int], Set[Tuple[int,int]]]:
    G = Graph()
    goals = {(n-1, n-1)}
    for i in range(n):
        for j in range(n):
            if (i,j)==(n-1,n-1):
                continue
            if i+1<n:
                G.add_edge((i,j),(i+1,j), risk=random.uniform(*rng))
            if j+1<n:
                G.add_edge((i,j),(i,j+1), risk=random.uniform(*rng))
    return G, (0,0), goals

def ucs_safest_path(G: Graph, start, goals):
    pq=[(0,start,[start])]
    visited={start:0}
    expanded=0
    t0=time.time()
    while pq:
        cost,u,path=heapq.heappop(pq)
        if u in goals:
            return path,cost,expanded,len(path)-1,round((time.time()-t0)*1000,3)
        for v,attrs in G.neighbors(u):
            if attrs['closed']:
                continue
            ncost=cost+attrs['risk']
            if ncost < visited.get(v,float('inf')):
                visited[v]=ncost
                heapq.heappush(pq,(ncost,v,path+[v]))
        expanded+=1
    return [],float('inf'),expanded,None,round((time.time()-t0)*1000,3)

def a_star_safest_path(G: Graph, start, goals, hop_dist):
    def h(n):
        return hop_dist.get(n,0)
    pq=[(h(start),0,start,[start])]
    visited={start:0}
    expanded=0
    t0=time.time()
    while pq:
        f,g,u,path=heapq.heappop(pq)
        if u in goals:
            return path,g,expanded,len(path)-1,round((time.time()-t0)*1000,3)
        for v,attrs in G.neighbors(u):
            if attrs['closed']:
                continue
            ng=g+attrs['risk']
            if ng < visited.get(v,float('inf')):
                visited[v]=ng
                heapq.heappush(pq,(ng+h(v),ng,v,path+[v]))
        expanded+=1
    return [],float('inf'),expanded,None,round((time.time()-t0)*1000,3)

def run():
    Gs,s,goals=build_small_graph()
    hop_s=compute_hop_dist(Gs,goals)
    print('Escenario pequeño:')
    for row in run_compare_algorithms({
        'UCS': lambda: ucs_safest_path(Gs,s,goals),
        'A*': lambda: a_star_safest_path(Gs,s,goals,hop_s)
    }):
        print(row)
    Gg,sg,ggoals=build_grid_graph()
    hop_g=compute_hop_dist(Gg,ggoals)
    print('\nCuadrícula 10x10:')
    for row in run_compare_algorithms({
        'UCS': lambda: ucs_safest_path(Gg,sg,ggoals),
        'A*': lambda: a_star_safest_path(Gg,sg,ggoals,hop_g)
    }):
        print(row)

if __name__ == '__main__':
    run()


UCS pequeño: ['A', 'B', 'D', 'E', 'G', 'I', 'J'] 24.0 10 6 0.021 ms
A* pequeño: ['A', 'B', 'D', 'E', 'G', 'I', 'J'] 24.0 10 6 0.015 ms
UCS 10x10: 19 58.0 99 18 0.127 ms


## Respuestas a Preguntas Orientadoras

**Reflexión crítica**
- En el grafo pequeño, UCS y A* siguen la misma ruta y expanden el mismo número de nodos; la heurística basada en saltos no aporta ventaja apreciable.
- En la cuadrícula 10x10, A* reduce ligeramente las expansiones al orientar la búsqueda, aunque el riesgo total se mantiene.
- Esto evidencia que la métrica de riesgo prioriza la seguridad sobre la distancia y que se requieren heurísticas más informativas para escalas mayores.
- Una mejora futura sería incorporar estimaciones probabilísticas del riesgo en tiempo real o heurísticas aprendidas.


## Análisis de complejidad de los algoritmos

- **BFS:** tiempo O(b^d) y espacio O(b^d).
- **IDS/DLS:** tiempo O(b^d) por reexpansiones y espacio O(b·d).
- **UCS:** en el peor caso tiempo y espacio O(b^d).
- **A*:** mismas cotas que UCS; con heurística eficaz reduce nodos expandidos.
- **A* con riesgo:** equivalente a UCS si h=0; con heurística de hops mantiene las mismas cotas.
